# Imports

In [ ]:
import numpy as np
import pandas as pd
import os
path = os.getcwd()
pd.set_option("max_columns",0)

In [ ]:
! pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer 

In [ ]:
! pip install nlu pyspark==3.0.2
import nlu

In [ ]:
! pip install hdbscan
import hdbscan

# Embedding

In [ ]:
features = pd.read_csv('./csv/features.csv', encoding = 'utf-8')


In [ ]:
stop_words=['of', 'use', 'few' , 'no' , 'loss', 'not', 'increase', 'decrease',\
    'ago'  , 'over', 'past', 'diminished' , 'last', 'prior', 'tp' , 'recent' ,\
    'normal', 'irregular', 'regulr', 'at', 'in',  'lack', 'other', 'due', 'with',\
    'associated', 'feeling', 'feels',  'subjective', 'induced', '-x-' , 'disorder',\
    'onces', 'symptoms', 'difficulty', 'unsuccessful', 'ineffective', 'an', 'a']
features['new_feature_text'] = features['feature_text'].str.replace('-', ' ')
features['new_feature_text'] = features['new_feature_text'].apply(lambda x: x.lower())
features['new_feature_text'] = features['new_feature_text'].str.replace(' or ', ', ')
features['new_feature_text'] = features['new_feature_text'].str.replace(r'[0-9]', ' ')
features.new_feature_text

In [ ]:
feature_un=features['new_feature_text'].unique()

# SentenceTransformer

In [ ]:
model = SentenceTransformer('bert-base-nli-mean-tokens')
embedding = model.encode(feature_un)

# GloVe

In [ ]:
pipe = nlu.load('glove')

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


In [ ]:
pipe.predict('He was suprised by the diversity of NLU')

sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]


,token,word_embedding_glove
0,He,"[0.12250000238418579, -0.05883299931883812, 0...."
0,was,"[0.1371700018644333, -0.5428699851036072, 0.19..."
0,suprised,"[-0.4543200135231018, 0.25780999660491943, -0...."
0,by,"[-0.20874999463558197, -0.11739999800920486, 0..."
0,the,"[-0.03819400072097778, -0.24487000703811646, 0..."
0,diversity,"[0.11683999747037888, 0.7225300073623657, 0.43..."
0,of,"[-0.15289999544620514, -0.24278999865055084, 0..."
0,NLU,"[0.4970400035381317, -0.013454999774694443, 0...."


In [ ]:
pipe.predict(list(feature_un))

,token,word_embedding_glove
0,family,"[0.4217900037765503, -0.09673000127077103, 0.1..."
0,history,"[0.21459999680519104, 0.7392100095748901, 0.79..."
0,of,"[-0.15289999544620514, -0.24278999865055084, 0..."
0,mi,"[-0.0731389969587326, -0.6566100120544434, 0.8..."
0,",","[-0.10767000168561935, 0.11052999645471573, 0...."
...,...,...
129,known,"[-0.3868800103664398, 0.0907140001654625, -0.1..."
129,illness,"[0.23713000118732452, -0.07117199897766113, 0...."
129,contacts,"[0.05528099834918976, -0.7338399887084961, -0...."
130,subjective,"[-0.5741199851036072, -0.11005999892950058, 0...."


# HDBSCAN only

In [ ]:
cluster = hdbscan.HDBSCAN(min_cluster_size = 5, metric = 'euclidean').fit(embedding)

In [ ]:
df = pd.DataFrame()

In [ ]:
df['labels']=cluster.labels_

In [ ]:
df['score']=np.count_nonzero(cluster.probabilities_ )

In [ ]:
df['features'] = feature_un

In [ ]:
df.labels.nunique()

3

In [ ]:
df[df['labels'] != -1]

,labels,score,features
0,0,87,Family-history-of-MI-OR-Family-history-of-myoc...
3,0,87,Intermittent-symptoms
5,0,87,No-hair-changes-OR-no-nail-changes-OR-no-tempe...
7,0,87,Shortness-of-breath
10,1,87,Few-months-duration
...,...,...,...
124,0,87,viral-symptoms-OR-rhinorrhea-OR-scratchy-throat
127,0,87,Family-history-of-migraines
128,0,87,Photophobia
129,0,87,No-known-illness-contacts
